In [53]:
import glob
import numpy as np
from PIL import *
import PIL.Image

In [54]:
filelist = glob.glob('train/cropped/glu/*.jpg')              #change '/'
glu = np.array([np.array(Image.open(fname)) for fname in filelist])
print(glu.shape)

(40, 497, 497, 3)


In [55]:
filelist = glob.glob('train/cropped/nglu/*.jpg')         #change '/'
nonglu = np.array([np.array(Image.open(fname)) for fname in filelist])
print(nonglu.shape)

(46, 497, 497, 3)


In [56]:
X = np.concatenate((glu,nonglu),axis=0)
print(X.shape)
print(X[1,:].shape)

(86, 497, 497, 3)
(497, 497, 3)


In [57]:
gluL = np.array([1 for i in range(40)])
nongluL = np.array([0 for i in range(46)])
y = np.concatenate((gluL,nongluL),axis=0)
print(y.shape)

(86,)


In [58]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(y_test)

(68, 497, 497, 3) (18, 497, 497, 3) (68,) (18,)
[0 1 0 1 1 0 1 1 1 0 0 0 0 0 0 0 1 0]


In [59]:
from keras.models import Sequential
from keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, Dropout,Dense,LeakyReLU
from keras.layers.core import Activation, Dense
model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', input_shape=X_train.shape[1:]))
# model.add(Activation('sigmoid'))
model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(0.10))
model.add(Conv2D(32, (3,3),padding='valid'))
# model.add(Activation('sigmoid'))
model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(0.10))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
# model.add(Activation('sigmoid'))
model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(0.10))
model.add(Conv2D(64, (3,3)))
# model.add(Activation('sigmoid'))
model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(0.10))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
# model.add(Activation('sigmoid'))
model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(0.10))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3)))
# model.add(Activation('sigmoid'))
model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(0.10))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3)))
# model.add(Activation('sigmoid'))
model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(0.10))
model.add(Conv2D(128, (3,3)))
# model.add(Activation('sigmoid'))
model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(0.10))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3)))
# model.add(Activation('sigmoid'))
model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(0.10))
model.add(Conv2D(256, (3,3)))
# model.add(Activation('sigmoid'))
model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(0.15))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, (1,1)))
# model.add(Activation('sigmoid'))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.10))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('sigmoid'))
# model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.10))
model.add(Dense(16))
model.add(Activation('sigmoid'))
# model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.10))
model.add(Dense(2, activation='sigmoid'))

In [60]:
model.output_shape

(None, 2)

In [61]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 497, 497, 32)      896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 497, 497, 32)      0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 495, 495, 32)      9248      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 495, 495, 32)      0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 247, 247, 32)      0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 245, 245, 32)      9248      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 245, 245, 32)      0         
__________

# DATA GENERATION

In [62]:
from keras.utils import np_utils
# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')
num_classes = 2
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator

datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                             rotation_range=15, width_shift_range=0.1,
                             height_shift_range=0.1,zoom_range=0.1,
                             fill_mode='constant', cval=0.0)

# print(datagen.fit(X_train))
print(y_train[0])


[1. 0.]


In [63]:
# from keras import optimizers
# sgd = optimizers.SGD(lr=0.0003, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='mean_squared_error', optimizer=sgd, metrics = ['accuracy'])
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [64]:
from keras.optimizers import *
# EPOCHS = 100
# INIT_LR = 1e-4
# BS = 32
# opt = Adam(lr=INIT_LR)
model.compile(optimizer=SGD(0.003),loss="binary_crossentropy",metrics=["accuracy"])
# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])
# train the network
print("[INFO] training network...")
H = model.fit_generator(datagen.flow(X_train, y_train, batch_size=1),
                        validation_data=(X_test, y_test), steps_per_epoch=len(X_train), epochs=100, verbose=1)

# model.fit(datagen.flow(X_train, y_train, batch_size=32),steps_per_epoch=len(X_train) / 32, epochs=10,validation_data=(X_test, y_test))

[INFO] training network...
Epoch 1/100
68/68 [==============================] - 71s 1s/step - loss: 0.7075 - acc: 0.5074 - val_loss: 0.6988 - val_acc: 0.4444
Epoch 2/100
68/68 [==============================] - 70s 1s/step - loss: 0.7102 - acc: 0.4926 - val_loss: 0.6775 - val_acc: 0.5833
Epoch 3/100
68/68 [==============================] - 70s 1s/step - loss: 0.6699 - acc: 0.5809 - val_loss: 0.6526 - val_acc: 0.6667
Epoch 4/100
68/68 [==============================] - 69s 1s/step - loss: 0.6566 - acc: 0.6103 - val_loss: 0.6547 - val_acc: 0.6667
Epoch 5/100
68/68 [==============================] - 70s 1s/step - loss: 0.6841 - acc: 0.5588 - val_loss: 0.6443 - val_acc: 0.6667
Epoch 6/100
68/68 [==============================] - 71s 1s/step - loss: 0.6806 - acc: 0.5735 - val_loss: 0.7132 - val_acc: 0.4167
Epoch 7/100
68/68 [==============================] - 70s 1s/step - loss: 0.6519 - acc: 0.6324 - val_loss: 0.7513 - val_acc: 0.3611
Epoch 8/100
68/68 [==============================] - 70s

Epoch 64/100
68/68 [==============================] - 71s 1s/step - loss: 0.4311 - acc: 0.8309 - val_loss: 0.5911 - val_acc: 0.7222
Epoch 65/100
68/68 [==============================] - 71s 1s/step - loss: 0.4444 - acc: 0.8235 - val_loss: 0.6003 - val_acc: 0.6667
Epoch 66/100
68/68 [==============================] - 70s 1s/step - loss: 0.4578 - acc: 0.7794 - val_loss: 0.6394 - val_acc: 0.6389
Epoch 67/100
68/68 [==============================] - 71s 1s/step - loss: 0.3904 - acc: 0.8750 - val_loss: 0.5827 - val_acc: 0.7500
Epoch 68/100
68/68 [==============================] - 71s 1s/step - loss: 0.4878 - acc: 0.7647 - val_loss: 0.5822 - val_acc: 0.6944
Epoch 69/100
68/68 [==============================] - 71s 1s/step - loss: 0.5532 - acc: 0.7426 - val_loss: 0.5732 - val_acc: 0.7222
Epoch 70/100
68/68 [==============================] - 71s 1s/step - loss: 0.4430 - acc: 0.8603 - val_loss: 0.5610 - val_acc: 0.7778
Epoch 71/100
68/68 [==============================] - 71s 1s/step - loss: 0.

In [65]:
score = model.evaluate(X_test, y_test, batch_size=16)
print(score)

18/18 [==============================] - 4s 219ms/step
[0.6255431473255157, 0.7222222222222222]


In [66]:
model.predict(X_test, batch_size=16)
model.predict_classes(X_test,batch_size=16)

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0])

In [67]:
import h5py
model.save_weights('final_weights.h5')